In [200]:
# %pip install plotly

In [223]:
import pandas as pd
import numpy as np
import csv
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

### Load Datasets

In [202]:
feat_df = pd.read_csv('FeaturesDataset.csv', parse_dates=['Date'])
print(feat_df.head())

   Store        Date  Temperature  Fuel_Price  MarkDown1  MarkDown2  \
0      1  05/02/2010        42.31       2.572        NaN        NaN   
1      1  12/02/2010        38.51       2.548        NaN        NaN   
2      1  19/02/2010        39.93       2.514        NaN        NaN   
3      1  26/02/2010        46.63       2.561        NaN        NaN   
4      1  05/03/2010        46.50       2.625        NaN        NaN   

   MarkDown3  MarkDown4  MarkDown5         CPI  Unemployment  IsHoliday  
0        NaN        NaN        NaN  211.096358         8.106      False  
1        NaN        NaN        NaN  211.242170         8.106       True  
2        NaN        NaN        NaN  211.289143         8.106      False  
3        NaN        NaN        NaN  211.319643         8.106      False  
4        NaN        NaN        NaN  211.350143         8.106      False  


In [203]:
sales_df = pd.read_csv('SalesDataset.csv', parse_dates=['Date'])
print(sales_df.head())

   Store  Dept        Date  Weekly_Sales  IsHoliday
0      1     1  05/02/2010      24924.50      False
1      1     1  12/02/2010      46039.49       True
2      1     1  19/02/2010      41595.55      False
3      1     1  26/02/2010      19403.54      False
4      1     1  05/03/2010      21827.90      False


### Overview on dataset structures
* replace NA with 0 because no markdown data means markdown not available
* markdown only available when there's promotions
* CPI and Unemployment will be inferenced using linear regression because it depends on the Store, Dept, and Date

In [205]:
print('-------FEATURES-------')
print(feat_df.info())

-------FEATURES-------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB
None


In [222]:
print(feat_df.isnull().sum())

Store             0
Date              0
Temperature       0
Fuel_Price        0
MarkDown1         0
MarkDown2         0
MarkDown3         0
MarkDown4         0
MarkDown5         0
CPI             585
Unemployment    585
IsHoliday         0
dtype: int64


In [206]:
print('-------SALES-------')
print(sales_df.info())

-------SALES-------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB
None


### Check for 0 in Markdown columns
* to make sure 0 does not exist

In [209]:
print('0 in features dataset\n', feat_df.where(feat_df == 0).sum())

0 in features dataset
 Store           0.0
Date              0
Temperature     0.0
Fuel_Price      0.0
MarkDown1       0.0
MarkDown2       0.0
MarkDown3       0.0
MarkDown4       0.0
MarkDown5       0.0
CPI             0.0
Unemployment    0.0
IsHoliday         0
dtype: object


### Replace NA in Markdown columns with 0

In [210]:
feat_df['MarkDown1'].fillna(0, inplace=True)
feat_df['MarkDown2'].fillna(0, inplace=True)
feat_df['MarkDown3'].fillna(0, inplace=True)
feat_df['MarkDown4'].fillna(0, inplace=True)
feat_df['MarkDown5'].fillna(0, inplace=True)
feat_df.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,True
2,1,19/02/2010,39.93,2.514,0.0,0.0,0.0,0.0,0.0,211.289143,8.106,False
3,1,26/02/2010,46.63,2.561,0.0,0.0,0.0,0.0,0.0,211.319643,8.106,False
4,1,05/03/2010,46.50,2.625,0.0,0.0,0.0,0.0,0.0,211.350143,8.106,False


### Merge tables
* Merge features and sales tables
* keys = Store, Date, IsHoliday
* to match the rows to store and dept at maximum
* no other data for matching store and dept

In [211]:
merged_df = pd.merge(feat_df, sales_df, how='left', left_on=['Store', 'Date', 'IsHoliday'], right_on=['Store', 'Date', 'IsHoliday'])
merged_df = merged_df.set_index('Date')
merged_df.head()

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Dept,Weekly_Sales
Date,,,,,,,,,,,,,
05/02/2010,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,1.0,24924.50
05/02/2010,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,2.0,50605.27
05/02/2010,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,3.0,13740.12
05/02/2010,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,4.0,39954.04
05/02/2010,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False,5.0,32229.38


### Structure of merged data

In [196]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 423325 entries, 05/02/2010 to 26/07/2013
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         423325 non-null  int64  
 1   Temperature   423325 non-null  float64
 2   Fuel_Price    423325 non-null  float64
 3   MarkDown1     423325 non-null  float64
 4   MarkDown2     423325 non-null  float64
 5   MarkDown3     423325 non-null  float64
 6   MarkDown4     423325 non-null  float64
 7   MarkDown5     423325 non-null  float64
 8   CPI           422740 non-null  float64
 9   Unemployment  422740 non-null  float64
 10  IsHoliday     423325 non-null  bool   
 11  Dept          421570 non-null  float64
 12  Weekly_Sales  421570 non-null  float64
dtypes: bool(1), float64(11), int64(1)
memory usage: 42.4+ MB


In [197]:
merged_df.isnull().sum()

Store              0
Temperature        0
Fuel_Price         0
MarkDown1          0
MarkDown2          0
MarkDown3          0
MarkDown4          0
MarkDown5          0
CPI              585
Unemployment     585
IsHoliday          0
Dept            1755
Weekly_Sales    1755
dtype: int64

### Check for null rows
* in Dept and Weekly_Sales
    * all of the rows with NA in CPI and Unemployment exist in the rows with NA in Dept and Weekly_Sales

In [220]:
null_rows = merged_df.loc[merged_df['Dept'].isnull() & merged_df['Weekly_Sales'].isnull()]
print(null_rows.count())
null_rows.isnull().sum()

Store           1755
Temperature     1755
Fuel_Price      1755
MarkDown1       1755
MarkDown2       1755
MarkDown3       1755
MarkDown4       1755
MarkDown5       1755
CPI             1170
Unemployment    1170
IsHoliday       1755
Dept               0
Weekly_Sales       0
dtype: int64


Store              0
Temperature        0
Fuel_Price         0
MarkDown1          0
MarkDown2          0
MarkDown3          0
MarkDown4          0
MarkDown5          0
CPI              585
Unemployment     585
IsHoliday          0
Dept            1755
Weekly_Sales    1755
dtype: int64

### Drop rows with null values
* in Dept and Weekly_Sales

In [199]:
merged_df.dropna(subset=['Dept', 'Weekly_Sales'], inplace=True)
merged_df.isnull().sum()

Store           0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
IsHoliday       0
Dept            0
Weekly_Sales    0
dtype: int64

### Write file to CSV

In [224]:
merged_df.to_csv('cleaned_file.csv')

### IGNORE!!!!

In [ ]:
# Unique values in each column
print('\nFeatures Unique Values')
for col in feat_df.columns:

    print(col, '\n', feat_df[col].unique())

In [ ]:
cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment']
plt.figure(figsize=(8, 5))
for col in cols:
    sns.histplot(data=feat_df, x=col)
    plt.title('Distribution of' + col)
    plt.xlabel(col)
    plt.ylabel("Values")
    plt.show()

In [180]:
def imputeNull(drop_col, impute_col, df, test_col):
    df.set_index('Date')
    print(df.info)
    lin_r = LinearRegression()
    temp_df = df.drop([drop_col], axis=1)

    test_df = temp_df.loc[temp_df[impute_col].isnull()==True]
    train_df = temp_df.loc[temp_df[impute_col].isnull()==False]
    y = train_df[impute_col]
    train_df = train_df.drop(impute_col, axis=1)
    lin_r.fit(train_df, y)
    test_df = test_df.drop(impute_col, axis=1)
    # print(test_df.isnull().sum(), train_df.isnull().sum())
    pred = lin_r.predict(test_df)
    test_df[impute_col] = pred

    train_df[impute_col] = y
    print(train_df.head())
    y = train_df[test_col]
    train_df.drop(test_col, axis=1)

    log_r = LogisticRegression()
    log_r.fit(train_df, y)

    LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
    y_test = test_df[test_col]
    test_df.drop(test_col, axis=1)
    pred = log_r.predict(test_df)
    print(metrics.accuracy_score(pred, y_test))





In [ ]:
cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment']
plt.figure(figsize=(8, 5))
for col in cols:
    sns.histplot(data=feat_df, x=col)
    plt.title('Distribution of' + col)
    plt.xlabel(col)
    plt.ylabel("Values")
    plt.show()

In [ ]:
imputeNull(['Unemployment', ], 'CPI', merged_df, 'Dept')

In [ ]:
print(merged_df.info())
lin_r = LinearRegression()
merged_df.head()
temp_df = merged_df.drop(['Unemployment'], axis=1)
testdf = temp_df.loc[temp_df['CPI'].isnull()==True]
traindf = temp_df.loc[temp_df['CPI'].isnull()==False]
print(traindf.head())
y = traindf['CPI']
traindf = traindf.drop("CPI",axis=1)
lin_r.fit(traindf,y)
testdf = testdf.drop("CPI",axis=1)
print(testdf.head())
pred = lin_r.predict(testdf)
testdf['CPI']= pred